# Software for managing ordinary store operations

In [ ]:
#--------------------------- README ---------------------------
"""
This software takes input in Italian and returns output in Italian.
For example: file names, exceptions, commands.
"""

#-------------------------------------------------------------------------------

import json # Given the hierarchical and nested structure of the data, and json flexibility
import os

#-------------------------------------------------------------------------------

# Files for inventory and sales. They are saved in the current working directory.
inventory_file="inventario.json"
sales_file="vendite.json"

#-------------------------------------------------------------------------------

def load_inventory():
  """
  This function loads the inventory from the file.
  If the file does not exist, it returns an empty list.
  If there are issues during loading, it raises an exception with a
  message and returns an empty list.
  """
  if os.path.exists(inventory_file):
    try:
      with open(inventory_file, "r", encoding="utf-8") as f:
        return json.load(f)
    except (json.JSONDecodeError, IOError):
      print("Errore nel caricamento file JSON per l'inventario.")
      return []
  else:
    return []

#-------------------------------------------------------------------------------

def save_inventory(inventory):
  """
  This function saves the inventory to the file (indentation = 4 spaces).
  """
  with open(inventory_file, "w", encoding="utf-8") as f:
    json.dump(inventory, f, indent=4)

#-------------------------------------------------------------------------------

def load_sales():
  """
  This function loads sales from the file.
  If the file does not exist, it returns an empty list.
  If there are issues during loading, it raises an exception with a message
  and returns an empty list.
  """
  if os.path.exists(sales_file):
    try:
      with open(sales_file, "r", encoding="utf-8") as f:
        return json.load(f)
    except (json.JSONDecodeError, IOError):
      print("Errore nel caricamento file JSON per le vendite.")
      return []
  else:
    return []

#-------------------------------------------------------------------------------

def save_sales(sales):
  """
  This function saves sales to the file (indentation = 4 spaces).
  """
  with open(sales_file, "w", encoding="utf-8") as f:
    json.dump(sales, f, indent=4)

#-------------------------------------------------------------------------------

def search_product_by_name(inventory, name):
  """
  This function searches for a product in the inventory by name
  (case-insensitive). If the product is in the inventory, it returns the
  dictionary; otherwise, it returns None.
  """
  for product in inventory:
    if product['nome'].lower()==name.lower(): # string conversion to lowercase
      return product
  return None

#-------------------------------------------------------------------------------

def add_product(inventory):
  """
  This function adds a product to the inventory, and if the product already
  exists, it takes the current value and increments the quantity
  with the new record.
  """
  name=input("Nome del prodotto: ").strip()  # It removes any extra spaces typed by mistake

  # Management of numeric input for quantity, purchase price, and selling price
  try:
    quantity=int(input("Quantità: "))
  except ValueError:
    print("Errore: la quantità deve essere un numero intero.")
    return

  # Verification that the quantity is not negative
  if quantity < 0:
    print("Errore: la quantità non può essere negativa.")
    return

  # Management of new record and existing product (updating the quantity)
  product=search_product_by_name(inventory, name)
  if product:
    product['quantità']+=quantity
    print(f"AGGIUNTO: {quantity} X {product['nome']}")
  else:
    try:
      purchase_price=float(input("Prezzo di acquisto: "))
      sale_price=float(input("Prezzo di vendita: "))
    except ValueError:
      print("Errore: il prezzo deve essere un numero decimale con il punto come separatore.")
      return

    # Verification that the prices are not negative
    if purchase_price < 0 or sale_price < 0:
      print("Errore: i prezzi non possono essere negativi.")
      return

    new_product={
      "nome": name,
      "quantità": quantity,
      "prezzo_acquisto": purchase_price,
      "prezzo_vendita": sale_price
    }
    inventory.append(new_product)
    print(f"AGGIUNTO: {quantity} X {name}")
  save_inventory(inventory)

#-------------------------------------------------------------------------------

def list_products(inventory):
  """
  This function returns a list of all products (with currency €) in the
  inventory that have a quantity greater than 0. If there are no products in
  the stock, it prints an appropriate message.
  """
  # A filter is implemented to show only products in stock with a quantity > 0.
  # If you want to see all products (including those with a quantity of 0), omit the marked line.
  if not inventory:
    print("Nessun prodotto nel magazzino.")
    return
  print("PRODOTTO\tQUANTITA'\tPREZZO")
  for product in inventory:
    if product['quantità'] > 0:  #  <<<<<---------------filter-----------------------
      # It shows the sale price while maintaining a fixed width to align the prints to the columns
      print(f"{product['nome']:<20}{product['quantità']:<12}€{product['prezzo_vendita']:<8}")

#-------------------------------------------------------------------------------

def register_sale(inventory, sales):
  """
  This function records a sale with one or more products. It checks that the
  products exist and that the quantity is sufficient for the transaction.
  If the sale is valid, it then subtracts the quantities from the inventory
  and records the successful sale transaction.
  """
  sale_products=[]
  while True:
    name=input("Nome del prodotto: ").strip()
    try:
      quantity=int(input("Quantità: "))
    except ValueError:
      print("Errore: la quantità deve essere un numero intero.")
      continue

    # It checks that the quantity is not negative
    if quantity < 0:
      print("Errore: la quantità non può essere negativa.")
      continue

    # Product existence and quantity sufficiency management
    product=search_product_by_name(inventory, name)
    if not product:
      print(f"Errore: il prodotto #{name} non esiste nel magazzino.")
      return  # It cancel the entire sales transaction
    if product['quantità'] < quantity:
      print(f"Errore: quantità insufficiente per il prodotto #{name}.")
      return  # It cancel the entire sales transaction

    # Adding the sold product to the list
    sale_products.append({
      "nome": product['nome'],
      "quantità": quantity,
      "prezzo_vendita": product['prezzo_vendita'],
      "prezzo_acquisto": product['prezzo_acquisto']
    })

    another=input("Aggiungere un altro prodotto ? (si/no): ").strip().lower()
    if another!="si":
      break

  # If all products are available, execute the sale by updating the inventory
  total=0.0
  for item in sale_products:
    # Update the quantity in inventory
    product=search_product_by_name(inventory, item['nome'])
    product['quantità']-=item['quantità']
    total+=item['quantità'] * item['prezzo_vendita']

  sale_record={"items": sale_products, "totale": total}
  sales.append(sale_record)
  save_inventory(inventory)
  save_sales(sales)

  # Print the sales summary
  print("VENDITA REGISTRATA")
  for item in sale_products:
    print(f" - {item['quantità']} X {item['nome']}: €{item['prezzo_vendita']}")
  # Format the total with two decimal places
  print(f"\nTotale: €{total:.2f}")

#-------------------------------------------------------------------------------

def show_profits(sales):
  """
  This function calculates and displays the gross profit and net profit.
  Gross profit = total sales amount; Net profit = gross profit - cost of the purchased products sold.
  """
  g_profit=0.0
  t_costs=0.0
  for sale in sales:
    g_profit+=sale['totale']
    for item in sale['items']:
      t_costs+=item['quantità'] * item['prezzo_acquisto']
  n_profit=g_profit - t_costs
  print(f"Profitto: lordo=€{g_profit:.2f} netto=€{n_profit:.2f}")

#-------------------------------------------------------------------------------

def show_help():
  """
  This function displays the available commands.
  """
  help_message=(
    "I comandi disponibili sono i seguenti:\n"
    "--> aggiungi: aggiungi un prodotto al magazzino\n"
    "--> elenca: elenca i prodotto in magazzino\n"
    "--> vendita: registra una vendita effettuata\n"
    "--> profitti: mostra i profitti totali\n"
    "--> aiuto: mostra i possibili comandi\n"
    "--> chiudi: esci dal programma"
  )
  print(help_message)

#-------------------------------------------------------------------------------

def invalid_command():
  """
  This function handles the case where the entered command is invalid,
  displaying a message and the menu of available commands.
  """
  print("Comando non valido")
  print("I comandi disponibili sono i seguenti:")
  print("--> aggiungi: aggiungi un prodotto al magazzino")
  print("--> elenca: elenca i prodotto in magazzino")
  print("--> vendita: registra una vendita effettuata")
  print("--> profitti: mostra i profitti totali")
  print("--> aiuto: mostra i possibili comandi")
  print("--> chiudi: esci dal programma")

#-------------------------------------------------------------------------------

##########################     MAIN LOOP      ###########################
inventory=load_inventory()
sales=load_sales()

cmd=None
while cmd!="chiudi":
  cmd=input("\nInserisci un comando: ").strip().lower()
  if cmd=="vendita":
    register_sale(inventory, sales)
  elif cmd=="profitti":
    show_profits(sales)
  elif cmd=="aggiungi":
    add_product(inventory)
  elif cmd=="elenca":
    list_products(inventory)
  elif cmd=="aiuto":
    show_help()
  elif cmd=="chiudi":
    print("Bye bye")
  else:
    invalid_command()


Inserisci un comando: aggiungi


KeyboardInterrupt: Interrupted by user